#该cell运行一次安装成功后无需再运行，不用在意ERROR，重启内核直接运行下一个cell
!pip install transformers -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --user datasets -i https://pypi.tuna.tsinghua.edu.cn/simple

In [1]:
import json
from Model import BertModel
from transformers import AutoTokenizer
import torch
from torch import nn
import re
from datasets import load_dataset
import os

In [2]:
class MyConfig(object):
    """Configuration class to store the configuration of a 'Model'
    """
    def __init__(self, vocab_size_or_config_json_file):
        with open(vocab_size_or_config_json_file, "r") as reader:
            json_config = json.loads(reader.read())
        for key, value in json_config.items():
            self.__dict__[key] = value

In [3]:
def load_file(file_path):
    with open(file_path) as f:
        test_set = f.readlines()

    test_set = [line.strip() for line in test_set]
    return test_set

In [4]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    my_config = MyConfig('./bert_config.json')
    my_model = BertModel(my_config, device=device)
    model_dic = torch.load('./pre-trained.bin', map_location=device)
    my_model.load_state_dict(model_dic, strict=False)
    my_model = my_model.to(device)
    my_model.eval()

    tokenizer = AutoTokenizer.from_pretrained("./tokenizer")
    X = load_file('./input/X_test.txt') 
    inputs = tokenizer(X, return_tensors="pt", padding=True).to(device)
    

    with torch.no_grad():
        logits = my_model(inputs['input_ids'])

    predicted_class_id = logits.argmax(-1).tolist()
    pred_txt = ['0' if x == 0 else '1' for x in predicted_class_id]

    with open('output/predict.txt', 'w') as g:
        g.write('\n'.join(pred_txt))


In [5]:
def zip_fun():
    path=os.getcwd()
    newpath=path+"/output/"
    os.chdir(newpath)
    os.system('zip prediction.zip predict.txt')
    os.chdir(path)

In [6]:
if __name__ == '__main__':
    main()
    zip_fun()

C:\Users\tommy\AppData\Local\Temp\ipykernel_6908\138369918.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dic = torch.load('./pre-trained.bin', map_location=devic